In [1]:
from google.colab import files #загрузка датасета
uploaded = files.upload()

Saving country_vaccinations.csv to country_vaccinations.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

In [48]:
df = pd.read_csv("country_vaccinations.csv")
df

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Argentina,ARG,2020-12-29,700.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1,Argentina,ARG,2020-12-30,NaN,NaN,NaN,NaN,15656.0,NaN,NaN,NaN,346.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
2,Argentina,ARG,2020-12-31,32013.0,NaN,NaN,NaN,15656.0,0.07,NaN,NaN,346.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
3,Argentina,ARG,2021-01-01,NaN,NaN,NaN,NaN,11070.0,NaN,NaN,NaN,245.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
4,Argentina,ARG,2021-01-02,NaN,NaN,NaN,NaN,8776.0,NaN,NaN,NaN,194.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Wales,NaN,2021-01-15,NaN,NaN,NaN,NaN,8468.0,NaN,NaN,NaN,2686.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1188,Wales,NaN,2021-01-16,NaN,NaN,NaN,NaN,8935.0,NaN,NaN,NaN,2834.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1189,Wales,NaN,2021-01-17,151938.0,151737.0,201.0,NaN,9403.0,4.82,4.81,0.01,2982.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1190,Wales,NaN,2021-01-18,162197.0,161932.0,265.0,10259.0,10123.0,5.14,5.14,0.01,3211.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [49]:
df.shape # размерность датафрэйма

(1192, 15)

In [50]:
df.isna().sum() # в каких стольбцах и сколько пропущенных значений

country                                   0
iso_code                                145
date                                      0
total_vaccinations                      442
people_vaccinated                       487
people_fully_vaccinated                1052
daily_vaccinations_raw                  611
daily_vaccinations                       56
total_vaccinations_per_hundred          442
people_vaccinated_per_hundred           487
people_fully_vaccinated_per_hundred    1052
daily_vaccinations_per_million           56
vaccines                                  0
source_name                               0
source_website                            0
dtype: int64

In [51]:
df.dtypes # какие типы данных в столбцах

country                                 object
iso_code                                object
date                                    object
total_vaccinations                     float64
people_vaccinated                      float64
people_fully_vaccinated                float64
daily_vaccinations_raw                 float64
daily_vaccinations                     float64
total_vaccinations_per_hundred         float64
people_vaccinated_per_hundred          float64
people_fully_vaccinated_per_hundred    float64
daily_vaccinations_per_million         float64
vaccines                                object
source_name                             object
source_website                          object
dtype: object

In [52]:
# есть ли категориальные признаки (объекты это категориальные признаки)
for i in range(len(df.dtypes)):
    if df.dtypes[i] == 'object':
        print(df.columns[i], df.dtypes[i])

country object
iso_code object
date object
vaccines object
source_name object
source_website object


In [53]:
 # удаляю столбцы в которыхбольше половины пропущенных значений
df = df.drop(['people_fully_vaccinated', 'daily_vaccinations_raw', 'people_fully_vaccinated_per_hundred'], axis=1)
df

,country,iso_code,date,total_vaccinations,people_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Argentina,ARG,2020-12-29,700.0,NaN,NaN,0.00,NaN,NaN,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1,Argentina,ARG,2020-12-30,NaN,NaN,15656.0,NaN,NaN,346.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
2,Argentina,ARG,2020-12-31,32013.0,NaN,15656.0,0.07,NaN,346.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
3,Argentina,ARG,2021-01-01,NaN,NaN,11070.0,NaN,NaN,245.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
4,Argentina,ARG,2021-01-02,NaN,NaN,8776.0,NaN,NaN,194.0,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Wales,NaN,2021-01-15,NaN,NaN,8468.0,NaN,NaN,2686.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1188,Wales,NaN,2021-01-16,NaN,NaN,8935.0,NaN,NaN,2834.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1189,Wales,NaN,2021-01-17,151938.0,151737.0,9403.0,4.82,4.81,2982.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1190,Wales,NaN,2021-01-18,162197.0,161932.0,10123.0,5.14,5.14,3211.0,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [55]:
# в числовых признаках, я заменяю пропуски на среднее значение
per=df['total_vaccinations'].mean()
df["total_vaccinations"].fillna(per, inplace = True)
per=df['people_vaccinated'].mean()
df["people_vaccinated"].fillna(per, inplace = True)
per=df['total_vaccinations_per_hundred'].mean()
df["total_vaccinations_per_hundred"].fillna(per, inplace = True)
per=df['people_vaccinated_per_hundred'].mean()
df["people_vaccinated_per_hundred"].fillna(per, inplace = True)
per=df['daily_vaccinations_per_million'].mean()
df["daily_vaccinations_per_million"].fillna(per, inplace = True)
per=df['daily_vaccinations'].mean()
df["daily_vaccinations"].fillna(per, inplace = True)
df

,country,iso_code,date,total_vaccinations,people_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Argentina,ARG,2020-12-29,700.000000,397863.720567,41080.478873,0.000000,2.069021,1335.222711,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1,Argentina,ARG,2020-12-30,499477.509333,397863.720567,15656.000000,2.152653,2.069021,346.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
2,Argentina,ARG,2020-12-31,32013.000000,397863.720567,15656.000000,0.070000,2.069021,346.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
3,Argentina,ARG,2021-01-01,499477.509333,397863.720567,11070.000000,2.152653,2.069021,245.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
4,Argentina,ARG,2021-01-02,499477.509333,397863.720567,8776.000000,2.152653,2.069021,194.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Wales,NaN,2021-01-15,499477.509333,397863.720567,8468.000000,2.152653,2.069021,2686.000000,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1188,Wales,NaN,2021-01-16,499477.509333,397863.720567,8935.000000,2.152653,2.069021,2834.000000,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1189,Wales,NaN,2021-01-17,151938.000000,151737.000000,9403.000000,4.820000,4.810000,2982.000000,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1190,Wales,NaN,2021-01-18,162197.000000,161932.000000,10123.000000,5.140000,5.140000,3211.000000,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [42]:
# удаляю строки с пустыми значениями
df.dropna()

,country,iso_code,date,total_vaccinations,people_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Argentina,ARG,2020-12-29,7.000000e+02,3.978637e+05,41080.478873,0.000000,2.069021,1335.222711,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1,Argentina,ARG,2020-12-30,4.994775e+05,3.978637e+05,15656.000000,2.152653,2.069021,346.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
2,Argentina,ARG,2020-12-31,3.201300e+04,3.978637e+05,15656.000000,0.070000,2.069021,346.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
3,Argentina,ARG,2021-01-01,4.994775e+05,3.978637e+05,11070.000000,2.152653,2.069021,245.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
4,Argentina,ARG,2021-01-02,4.994775e+05,3.978637e+05,8776.000000,2.152653,2.069021,194.000000,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...
1149,United States,USA,2021-01-15,1.227918e+07,1.059587e+07,798707.000000,3.710000,3.200000,2413.000000,"Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://covid.cdc.gov/covid-data-tracker/#vacc...
1150,United States,USA,2021-01-16,4.994775e+05,3.978637e+05,811670.000000,2.152653,2.069021,2452.000000,"Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://covid.cdc.gov/covid-data-tracker/#vacc...
1151,United States,USA,2021-01-17,4.994775e+05,3.978637e+05,824632.000000,2.152653,2.069021,2491.000000,"Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://covid.cdc.gov/covid-data-tracker/#vacc...
1152,United States,USA,2021-01-18,4.994775e+05,3.978637e+05,837595.000000,2.152653,2.069021,2530.000000,"Moderna, Pfizer/BioNTech",Centers for Disease Control and Prevention,https://covid.cdc.gov/covid-data-tracker/#vacc...
